##### 1.Сделайте краткий обзор любой научной работы, посвящённой алгоритму для object detection, не рассматривавшемуся на уроке. Проведите анализ: чем отличается выбранная вами архитектура нейронной сети от других? В чём плюсы и минусы данной архитектуры? Какие могут возникнуть трудности при применении этой архитектуры на практике?

Архитектура Faster R-CNN объединяет Region Proposal Network (RPN) и Fast R-CNN в единую сеть
путем совместного использования их свертывающих характеристик, используя недавно популярную
терминологию нейронных сетей "механизмы внимания". RPN используется для генерации гипотез,
получая на вход карту признаков изначального изображения. Сеть Faster R-CNN строится на одних и тех
же весах с помощью следующего итеративного обучения:

1. Тренировка RPN сети. Свёрточные слои инициализируются весами, натренированными на
ImageNet. Дообучаем на задаче определения регионов с каким-либо классом (уточнением
класса занимается часть Fast R-CNN).
2. Тренировка Fast R-CNN сети. Свёрточные слои инициализируются весами, натренированными
на ImageNet. Дообучаем, используя гипотезы об объектах с помощью RPN сети, обученной в п.1.
В этот раз задачей обучения является уточнение координат и определение конкретного класса
объекта.
После выполнения п.2 мы имеем две нейронных сети с разными весами для свёрточных слоёв.
3. Обучаем только RPN часть, используя веса из п.2 (слои, идущие до RPN сети, принадлежащие
feature extractor, замораживаются и никак не изменяются).
4. Обучаем слои для Fast R-CNN, используя веса из п.3 (то есть, уже более точно настроенный RPN),
(остальные веса – идущие ранее или относящиеся к RPN — заморожены).
Таким образом, архитектура Faster R-CNN отличается от других архитектур тем, что генерация
гипотез выполняется с помощью отдельно дифференцируемого модуля и в архитектуре реализовано
итеративное обучение.
Преимуществом архитектуры Faster R-CNN являются быстрота и точность. Архитектура Faster RCNN справляется немного хуже с локализацией. RPN может быть трудно работать с объектами разных
масштабов. RPN имеет фиксированное приемное поле. Таким образом, малые объекты могут занимать
очень небольшую часть рецептивного поля или, если объекты очень большие, то рецептивное поле
будет содержать только часть объекта. Проблему можно решать, тренируя множество RPN для
различных масштабов: каждая часть RPN будет принимать различные свёрточные слои или наборы
слоёв в качестве входа, так что приемное поле будет разного размера. Это значительно улучшит
одновременное обнаружение малых и крупных объектов моделью Faster R-CNN.

#### Особенности тренировки

Фактически необходимо натренировать две сети: Fast R-CNN для классификации и уточнения координат объектов, RPN для генерации претендентов. При этом свёрточные слои должны быть общими для этих двух сетей (собственно, всё и затевалось ради того, чтобы суметь использовать одну карту особенностей для решения обеих задач).

Авторы предлагают несколько возможных подходов к тренировке, однако, результаты, представленные в статье, получены при помощи тренировки по следующей схеме:

 1. Тренируем сеть для генерации претендентов. Для этого инициализируем свёрточные слои весами натренированными на ImageNet и доуточняем веса свёрточных слоёв и слоёв RPN части 
 
 2. Тренируем сеть Fast R-CNN сеть. Свёрточные слои инициализируем весами от сети, натренированной на ImageNet и дотренировываем. При этом претенденты для изображения генерируем при помощи сети полученной в п.1.

 3. Важно, что после выполнения п.2 мы имеем две нейронных сети с разными весами для свёрточных слоёв.

 4. Берем свёрточные слои от Fast R-CNN, которую натренировали в п.2, переносим их в RPN сеть и дотренировываем её. При этом веса свёрточных слоёв фиксируются, а тренируются только веса слоёв rpn_conv, rpn_cls_score и rpn_bbox_pred.

5. Дотренировываем Fast R-CNN с учетом изменившегося генератора претендентов. При этом опять не меняем веса свёрточных слоёв, и тренируем только слои специфичные для Fast R-CNN.

На этом тренировка заканчивается